In [1]:
import requests
import json
import csv
import pprint
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'requests'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/Google_Map_review_data.csv')

In [ ]:
api_key_ocm = '8b74f800-157a-4173-b0ff-c8d4a565081b' #'78770855-dee3-4eff-b188-d3273394fe0f'
location = df[['lat','lng']]
location = location.drop_duplicates().reset_index(drop = True)
latitudes = location.lat.astype(float)
longitudes = location.lng.astype(float)
# times = df.time
# country_code = 'US'
ids = []
# empty dataframe to append data
ocm_data = pd.DataFrame()
units = "metric"
count = 10
distanceunit = "km"
output="json"

## loop through each major city and collect the details from opemchargemap. Same for the states
for i in range(len(latitudes)):
  print(i)
  latitude = latitudes[i]
  longitude = longitudes[i]

  distance = 2000 # In meters
  countrycode="US"

  # Specify the API endpoint and parameters
  api_url = f"https://api.openchargemap.io/v3/poi/?key={api_key_ocm}&distance={distance}&longitude={longitude}&latitude={latitude}&countrycode=US&maxresults={count}&compact=false&includecomments=true&output={output}&verbose=false"

  response = requests.get(api_url).json()

  if len(response)==0:
    ids.append(i); # save ids for charging station that have review but don't have record in OCM


  df_nested=pd.json_normalize(response)
  #df_nested['time'] = times[i]
  df_nested['Address'] = df_nested['AddressInfo.AddressLine1'] +', ' + df_nested['AddressInfo.Town'] +', ' +  df_nested['AddressInfo.StateOrProvince'] +' ' + df_nested['AddressInfo.Postcode'] +', ' + df_nested['AddressInfo.Country.ISOCode']
  df_nested['Address'] = df_nested['Address'].astype(str)
  df_nested['match'] = df_nested['Address'].apply(lambda x: any(x in address for address in df['address']))
  df_nested = df_nested[df_nested['match']].reset_index(drop = True)
  for j in range(len(df_nested)):
    if len(df_nested['Connections'][j])>0:
      temp = pd.DataFrame(df_nested['Connections'][j][0])
      temp = temp.dropna().reset_index(drop = 'True')
      row = temp.iloc[1:2].reset_index(drop = 'True')
      temp = pd.concat([row,df_nested],axis = 1, join='inner')
      ocm_data = ocm_data.loc[:, ~ocm_data.columns.duplicated()]
      temp = temp.loc[:, ~temp.columns.duplicated()]
      ocm_data = pd.concat([ocm_data, temp], axis=0, ignore_index=True)


In [ ]:
#ocm_data.to_csv('/content/drive/MyDrive/ColabNotebooks/Google_Map_review_data_combined.csv', index=False)
ocm_data = ocm_data.drop_duplicates().reset_index(drop = True)
ocm_data.to_csv('/content/drive/MyDrive/ColabNotebooks/Google_Map_review_data_combined.csv', index=False)